In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

In [ ]:
def gompertz(t,m,A,l):
    return A * np.exp(-np.exp(m*np.e/A*(l-t) + 1))

In [ ]:
# kbase = GPy.kern.RBF(1,name='base')
# kbio = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1)])
# ktech = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.RBF(1,name='bio', variance=.1), GPy.kern.RBF(1,name='tech', variance=.05)])

In [ ]:
nbatch = 3
nrep = 3
ntot = nbatch * nrep

x = np.zeros((50*ntot,3))
x[:,0] = np.tile(np.linspace(-1,1), ntot)
x[:,1] = np.repeat(np.arange(nbatch), nrep*50)
x[:,2] = np.repeat(np.arange(nbatch*nrep), 50)

In [ ]:
baseVariance, batchVariance, repVariance = .01, .01, .005

kbase = GPy.kern.RBF(1,name='base',variance=baseVariance)
kbatch = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='batch', variance=batchVariance), index_dim=-2);
krep = GPy.kern.IndependentOutputs(GPy.kern.RBF(1,name='replicate', variance=repVariance))

In [ ]:
plt.imshow(kbatch.K(x))

plt.figure()
plt.imshow(krep.K(x))

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(221)
plt.imshow(kbase.K(x))

plt.subplot(222)
plt.imshow(kbatch.K(x) + kbase.K(x))

plt.subplot(223)
plt.imshow(krep.K(x) + kbase.K(x))

plt.subplot(224)
plt.imshow(krep.K(x) + kbatch.K(x) + kbase.K(x))

In [ ]:
def generateSample(mu, cov, nugget, length=50):
    noise = np.eye(mu.shape[0])*nugget
    
    return scipy.stats.multivariate_normal.rvs(mu,cov+noise).reshape((mu.shape[0]/length,length)).T

In [ ]:
cov = np.zeros((x.shape[0]*4, x.shape[0]*4))

# cov = kbase.K(np.tile(x.T,4).T) #+ batchVariance + repVariance

print cov.shape

# cov[:,:x.shape[0]] = np.tile(kbase.K(x), 4).T
# cov[:x.shape[0],:] = np.tile(kbase.K(x), 4)
# cov[:x.shape[0],:] = kbase.K(x)
cov[x.shape[0]:2*x.shape[0],x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance
cov[x.shape[0]:2*x.shape[0],3*x.shape[0]:] += kbatch.K(x) #- batchVariance
cov[3*x.shape[0]:,x.shape[0]:2*x.shape[0]] += kbatch.K(x) #- batchVariance

cov[2*x.shape[0]:3*x.shape[0],2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance
cov[2*x.shape[0]:3*x.shape[0],3*x.shape[0]:] += krep.K(x) #- repVariance
cov[3*x.shape[0]:,2*x.shape[0]:3*x.shape[0]] += krep.K(x) #- repVariance

cov[3*x.shape[0]:,3*x.shape[0]:] += kbatch.K(x) + krep.K(x) # - repVariance-  batchVariance

plt.imshow(cov)

In [ ]:
np.random.seed(1)

f = gompertz(x[:,0]+1, 2, 1, .4)
s = generateSample(np.tile(f, 4), cov, sigma)

plt.figure(figsize=(10,10))
for i in range(4):
    plt.subplot(2,2,i+1)
    
    plt.plot(f[:50],c='k', lw=3)
    
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(s[:,ntot*i:ntot*(i+1)][:,j],color='C%d'%k,alpha=.6);

In [ ]:
plt.figure(figsize=(10,10))
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.plot([0, 50], [0,0], c='k', lw=3)
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(s[:,ntot*i:ntot*(i+1)][:,j]-f[:50],color='C%d'%k);

In [ ]:
plt.figure(figsize=(10,10))

sigma = .001

plt.subplot(221)
cov = kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

plt.subplot(222)
cov = kbatch.K(x) + kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

plt.subplot(223)
cov = krep.K(x) + kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

plt.subplot(224)
cov = kbatch.K(x) + krep.K(x) + kbase.K(x)
plt.plot(generateSample(gompertz(x[:,0]+1, 2, 1, .4), cov, sigma))

In [ ]:
sigma = .001
# noise = np.eye(50*ntot)*sigma

f = gompertz(x[:,0]+1, 2, 1, .4)

np.random.seed(1)

plt.figure(figsize=(12,4))
for i,kern in enumerate([None, kbio, ktech]):

    mu = np.zeros(x.shape[0])
    
    if not kern is None:
        cov = kern.K(x)
        f += scipy.stats.multivariate_normal.rvs(mu,cov)
        
    extraVariance = sum([0, bioVariance, techVariace][i+1:])
    noise = np.eye(50*ntot)*(sigma+extraVariance)
    
    s = scipy.stats.multivariate_normal.rvs(f,noise).reshape((ntot,50)).T
    
    plt.subplot(1,3,i+1)
    
#     if i > 0:
#         for j,z in enumerate(x[::50,i]):
#             k = np.unique(x[:,i]).tolist().index(z)
#             plt.plot(s[:,j],color='C%d'%k);
#     else:
#         plt.plot(s,c='k');
    for j,z in enumerate(x[::50,1]):
            k = np.unique(x[:,1]).tolist().index(z)
            plt.plot(s[:,j],color='C%d'%k);
            
    plt.ylim(-.25,1.3)
        
plt.savefig("figures/simulated-data.pdf",bbox_inches='tight')